This set of code is run on the raw reads from the Illumina sequencer:

In [2]:
# /bin/sh 
# ----------------Parameters---------------------- #
#$  -S /bin/sh
#$ -q sThC.q
#$ -l mres=8G,h_data=8G,h_vmem=8G
#$ -l cpu_arch='haswell|skylake'
#$ -cwd
#$ -j y
#$ -N fastqc_unrefined_v4
#$ -o ../logs/fastqc_unrefined_v4.log
#
# ----------------Modules------------------------- #
module load bioinformatics/fastqc
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
#
fastqc ../data/results/*.fastq \
--outdir=../data/results
#
echo = `date` job $JOB_NAME done

SyntaxError: invalid syntax (<ipython-input-2-a1136750dae3>, line 13)

This script does what it needs to, but kind of indiscriminately dumps everything in control. Kind of want to add a step where it puts everything together in a nice little box. The next step is to trim reads using Trimgalore:

In [3]:
#need to write this script

### Read filtering
To remove host reads, I have used Bowtie2's "build" function to create an index of the genome that was available, then aligned those using bowtie2. Then you separate mapped and unmapped reads using samtools, which can also separate them into different files and re-purpose them into fastq files. 

In [4]:
# /bin/sh 
# ----------------Parameters---------------------- #
#$  -S /bin/sh
#$ -pe mthread 4
#$ -q sThC.q
#$ -l mres=32G,h_data=8G,h_vmem=8G
#$ -l cpu_arch='haswell|skylake'
#$ -cwd
#$ -j y
#$ -N bowtie2-removehost
#$ -o ../logs/bowtie2-removehost.log
#
# ----------------Modules------------------------- #
module load bioinformatics/bowtie2
module load bioinformatics/samtools
#
# ----------------Your Commands------------------- #
#
echo + `date` job $JOB_NAME started in $QUEUE with jobID=$JOB_ID on $HOSTNAME
echo + NSLOTS = $NSLOTS
#
for f in Coral1 Coral2 Coral4 Coral5
do
bowtie2 -p 8 -x ../reference/bowtie-index/M.cavernosa -1 ../../00_raw/"$f"_R1_001_val_1.fq -2 ../../00_raw/"$f"_R2_001_val_2.fq -S ../data/working/"$f"_mapped_and_unmapped.sam
samtools view -bS ../data/working/"$f"_mapped_and_unmapped.sam > ../data/working/"$f"_mapped_and_unmapped.bam
samtools view -b -f 12 -F 256 ../data/working/"$f"_mapped_and_unmapped.bam > ../data/working/"$f"_bothReadsUnmapped.bam
samtools sort -n -m 5G -@ 2 ../data/working/"$f"_bothReadsUnmapped.bam -o ../data/working/"$f"_bothReadsUnmapped_sorted.bam
samtools fastq -@ 8 ../data/working/"$f"_bothReadsUnmapped_sorted.bam \
    -1 "$f"_host_removed_R1.fastq.gz \
    -2 "$f"_host_removed_R2.fastq.gz \
    -0 /dev/null -s /dev/null -n
done
#
echo = `date` job $JOB_NAME done


SyntaxError: invalid syntax (<ipython-input-4-8666ed064be9>, line 14)

This leaves you with trimmed, filtered reads. After this, you should run FastQC again. Be mindful that you change its logging location and the name of the directory, OR IT WILL OVERWRITE THE OTHER FILES